### Introduction to Marmot

Last updated: 01/21/2020

In [1]:
from marmot import Environment, Agent, process

Marmot is an extension of [SimPy](https://simpy.readthedocs.io/en/latest/) that makes it easier to build agent-based process models. The three main components of Marmot are the Environment class, Agent class and the @process decorator:

- **Environment**: This class is very similar to the simpy.Environment class with a few additional features
- **Agent**: This class and it's subclasses represent discrete agents that must be registered with an environment before they can perform tasks
- **@process**: This decorator is used it denote a simpy.Process and verify that the configured agent is registered with an Environment. It also handles most of the boilerplate code that is required by SimPy to call nested processes.

#### Environment

In [3]:
# The marmot.Environment extends simpy.Environment to include the following functionality:
# - Registering an Agent with the environment (required to use the @process decorator)
# - Tracking which registered agent is submitting a process to the underlying Environment._queue
# - State time series data that can be used to constrain Agent tasks from running unless a set of conditions are met
# - Simple logging of agent tasks (to be expanded on in future releases)


# To create an Environment without state data:
env = Environment(name="Test Environment")

print(f"Environment time: {env.now}")      # The simpy.Environment API is maintained
print(f"Environment queue: {env._queue}")  # There are no events scheduled at creation

# Additional properties:
print(f"Active agents: {env.active_agents}")        # There are no agents registered with the environment at creation
print(f"Scheduled agents: {env.scheduled_agents}")  # No scheduled agents
print(f"Logs: {env.logs}")                          # No logs

Environment time: 0
Environment queue: []
Active agents: []
Scheduled agents: []
Logs: []


In [4]:
env.timeout(10)  # Schedule a timeout of length 10
env._queue       # Notice the "None" at the end of the Timeout(10) object.
                 # This represents which agent this event is attributed to.
                 # In this case, the timeout isn't attributed to an agent (more later).

[(10, 1, 0, <Timeout(10) object at 0x10d587df0>, None)]

#### Agent

In [5]:
# The marmot.Agent class represents a discrete Agent in the process model. A few notes:
# - An Agent must be registered with an Environment before it can perform any tasks
# - Each Agent registered with an Environment must have a unique name


# Reset env, create an agent and register it
env = Environment(name="Test Environment")
agent = Agent("Test Agent")
env.register(agent)

print(f"Active agents: {env.active_agents}")        # "Test Agent" is now registered with "env".

Active agents: [Test Agent]


In [6]:
# "Test Agent" can now submit tasks to "env"

agent.task("Run", duration=10)  # Submitting a task with name="Run" that will last for 10 units

print(f"Scheduled agents: {env.scheduled_agents}")  # "Test Agent" is now scheduled
print(f"Environment queue: {env._queue}")           # The event in env._queue is attributed to "Test Agent"

Scheduled agents: [Test Agent]
Environment queue: [(0, 0, 0, <Initialize() object at 0x11aff9130>, Test Agent)]


In [7]:
env.run()  # Run the current environment.
env.logs   # The "task" method automatically submits a log of the action.
           # The "time" represents when the task was completed.

[{'agent': 'Test Agent',
  'action': 'Run',
  'duration': 10.0,
  'level': 'ACTION',
  'time': 10}]

In [8]:
agent.task("Rest", 60, status="Out of breath")  # kwargs passed to task are automatically passed to the log.
env.run()
env.logs

[{'agent': 'Test Agent',
  'action': 'Run',
  'duration': 10.0,
  'level': 'ACTION',
  'time': 10},
 {'status': 'Out of breath',
  'agent': 'Test Agent',
  'action': 'Rest',
  'duration': 60.0,
  'level': 'ACTION',
  'time': 70}]

In [9]:
# Registration Errors:
# Another agent with name "Test Agent" can't be registered until "agent" is deregistered.
agent2 = Agent("Test Agent")
env.register(agent2)

RegistrationConflict: 'Test Environment' already has a registered agent with name 'Test Agent'.

In [10]:
# agent2 can't submit a task until it is registered with an Environment
agent2.task("Run", 10)

AgentNotRegistered: Agent 'Test Agent' is not registered to an environment.

#### @process

In [13]:
# The "process" decorator enables a lot of the functionality above. It does a few things:
# - Verifies that the submitting Agent is registered (else raises AgentNotRegistered)
# - Verifies that the submitting Agent isn't already scheduled (else raises AgentAlreadyScheduled)
# - Wraps the input function in "env.process(func(...))".
# -- This is a requirement of SimPy. All generators are treated as a simpy.Process and as such, any generator
#    called within another generator must be submitted with "env.process(func(...))".


# The "process" decorator can be used to build up a series of tasks.
# It takes the first argument to the function as the Agent and performs the checks outlined above.
# This design allows it to be used within and outside of an instance of Agent:

# Outside of an Agent instance
@process
def perform_series_of_tasks(agent, a, b, c):
    
    yield agent.task("Task A", a)
    yield agent.task("Task B", b)
    yield agent.task("Task C", c)
    

# Within an Agent instance
class TestAgent(Agent):
    
    def __init__(self, name):
        super().__init__(name)

    @process
    def perform_series_of_tasks(self, a, b, c):

        yield self.task("Task A", a)
        yield self.task("Task B", b)
        yield self.task("Task C", c)
    

In [14]:
# Reset env, create an agent and register it
env = Environment(name="Test Environment")
agent = Agent("Test Agent")
env.register(agent)

_ = perform_series_of_tasks(agent, 2, 4, 6)
env._queue
env.run()
env.logs

[{'agent': 'Test Agent',
  'action': 'Task A',
  'duration': 2.0,
  'level': 'ACTION',
  'time': 2},
 {'agent': 'Test Agent',
  'action': 'Task B',
  'duration': 4.0,
  'level': 'ACTION',
  'time': 6},
 {'agent': 'Test Agent',
  'action': 'Task C',
  'duration': 6.0,
  'level': 'ACTION',
  'time': 12}]

In [15]:
# This time with TestAgent.perform_series_of_tasks
env = Environment(name="Test Environment")
agent = TestAgent("Test Agent")
env.register(agent)

agent.perform_series_of_tasks(2, 4, 6)
env._queue
env.run()
env.logs

[{'agent': 'Test Agent',
  'action': 'Task A',
  'duration': 2.0,
  'level': 'ACTION',
  'time': 2},
 {'agent': 'Test Agent',
  'action': 'Task B',
  'duration': 4.0,
  'level': 'ACTION',
  'time': 6},
 {'agent': 'Test Agent',
  'action': 'Task C',
  'duration': 6.0,
  'level': 'ACTION',
  'time': 12}]